# Set up environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.7.13


In [3]:
# install environment
!pip install contextily xarray zarr aiohttp requests pystac pystac-client planetary-computer scikit-learn fsspec shapely rioxarray stackstac xarray-spatial

#Step 1
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y
#Step 4
!apt-get install python-numpy python-scipy -y
#Step 5
import gdal 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 153 kB 12.5 MB/s 
     |████████████████████████████████| 1.1 MB 69.4 MB/s 
     |████████████████████████████████| 137 kB 70.5 MB/s 
     |████████████████████████████████| 140 kB 64.9 MB/s 
     |████████████████████████████████| 47 kB 4.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 10.9 MB 29.3 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 6.2 MB 53.9 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
     |████████████████████████████████| 271 kB 63.2 MB/s 
     |████████████████████████████████| 63 kB 80 kB/s 
     |████████████████████████████████| 11.1 MB 73.9 MB/s 
     |████████████████████

## Load in dependencies

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Data science
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Geospatial
import contextily as cx
import xarray as xr
import zarr # Not referenced, but required for xarray

# Import Planetary Computer tools
import fsspec
import pystac

# Other
import os
import zipfile
from itertools import cycle
import pickle

# Path to data folder with provided material
data_path = './2022-Better-Working-World-Data-Challenge/notebooks/'

In [5]:
# if not os.path.exists(data_path+'training_data/'):
#     os.mkdir(data_path+'training_data/')
#     with zipfile.ZipFile(data_path+'GBIF_training_data.zip', 'r') as zip_ref:  # open zip
#         zip_ref.extractall(data_path+'training_data/')
        
def filter_bbox(frogs, bbox):
    frogs = frogs[lambda x: 
        (x.decimalLongitude >= bbox[0]) &
        (x.decimalLatitude >= bbox[1]) &
        (x.decimalLongitude <= bbox[2]) &
        (x.decimalLatitude <= bbox[3])
    ]
    return frogs

def get_frogs(file, year_range=None, bbox=None):
    """Returns the dataframe of all frog occurrences for the bounding box specified."""
    columns = [
        'gbifID','eventDate','country','continent','stateProvince',
        'decimalLatitude','decimalLongitude','species'
    ]
    country_names = {
        'AU':'Australia', 'CR':'Costa Rica', 'ZA':'South Africa','MX':'Mexico','HN':'Honduras',
        'MZ':'Mozambique','BW':'Botswana','MW':'Malawi','CO':'Colombia','PA':'Panama','NI':'Nicaragua',
        'BZ':'Belize','ZW':'Zimbabwe','SZ':'Eswatini','ZM':'Zambia','GT':'Guatemala','LS':'Lesotho',
        'SV':'El Salvador', 'AO':'Angola', np.nan:'unknown or invalid'
    }
    continent_names = {
        'AU':'Australia', 'CR':'Central America', 'ZA':'Africa','MX':'Central America','HN':'Central America',
        'MZ':'Africa','BW':'Africa','MW':'Africa','CO':'Central America','PA':'Central America',
        'NI':'Central America','BZ':'Central America','ZW':'Africa','SZ':'Africa','ZM':'Africa',
        'GT':'Central America','LS':'Africa','SV':'Central America','AO':'Africa', np.nan:'unknown or invalid' 
    }
    frogs = (
        pd.read_csv(data_path+'training_data/occurrence.txt', sep='\t', parse_dates=['eventDate'])
        .assign(
            country =  lambda x: x.countryCode.map(country_names),
            continent =  lambda x: x.countryCode.map(continent_names),
            species = lambda x: x.species.str.title()
        )
        [columns]
    )
    if year_range is not None:
        frogs = frogs[lambda x: 
            (x.eventDate.dt.year >= year_range[0]) & 
            (x.eventDate.dt.year <= year_range[1])
        ]
    if bbox is not None:
        frogs = filter_bbox(frogs, bbox)
    return frogs

In [6]:
def get_terraclimate(bbox, metrics, time_slice=None, assets=None, features=None, interp_dims=None, verbose=True):
    """Returns terraclimate metrics for a given area, allowing results to be interpolated onto a larger image.
    
    Attributes:
    bbox -- Tuple of (min_lon, min_lat, max_lon, max_lat) to define area
    metrics -- Nested dictionary in the form {<metric_name>:{'fn':<metric_function>,'params':<metric_kwargs_dict>}, ... }
    time_slice -- Tuple of datetime strings to select data between, e.g. ('2015-01-01','2019-12-31')
    assets -- list of terraclimate assets to take
    features -- list of asset metrics to take, specified by strings in the form '<asset_name>_<metric_name>'
    interp_dims -- Tuple of dimensions (n, m) to interpolate results to
    """
    min_lon, min_lat, max_lon, max_lat = bbox
    
    collection = pystac.read_file("https://planetarycomputer.microsoft.com/api/stac/v1/collections/terraclimate")
    asset = collection.assets["zarr-https"]
    store = fsspec.get_mapper(asset.href)
    data = xr.open_zarr(store, **asset.extra_fields["xarray:open_kwargs"])
    
    # Select datapoints that overlap region
    if time_slice is not None:
        data = data.sel(lon=slice(min_lon,max_lon),lat=slice(max_lat,min_lat),time=slice(time_slice[0],time_slice[1]))
    else:
        data = data.sel(lon=slice(min_lon,max_lon),lat=slice(max_lat,min_lat))
    if assets is not None:
        data = data[assets]
    print('Loading data') if verbose else None
    data = data.rename(lat='y', lon='x').to_array().compute()
        
    # Calculate metrics
    combined_values = []
    combined_bands = []
    for name, metric in metrics.items():
        print(f'Calculating {name}') if verbose else None
        sum_data = xr.apply_ufunc(
            metric['fn'], data, input_core_dims=[["time"]], kwargs=metric['params'], dask = 'allowed', vectorize = True
        ).rename(variable='band')
        xcoords = sum_data.x
        ycoords = sum_data.y
        dims = sum_data.dims
        combined_values.append(sum_data.values)
        for band in sum_data.band.values:
            combined_bands.append(band+'_'+name)
        
    # Combine metrics
    combined_values = np.concatenate(
        combined_values,
        axis=0
    )
    combined_data = xr.DataArray(
        data=combined_values,
        dims=dims,
        coords=dict(
            band=combined_bands,
            y=ycoords,
            x=xcoords
        )
    )    

    # Take relevant bands:
    combined_data = combined_data.sel(band=features)
    
    if interp_dims is not None:
        print(f'Interpolating image') if verbose else None
        interp_coords = (np.linspace(bbox[0], bbox[2], interp_dims[0]), np.linspace(bbox[1], bbox[3], interp_dims[1]))
        combined_data = combined_data.interp(x=interp_coords[0], y=interp_coords[1], method='nearest', kwargs={"fill_value": "extrapolate"})
    
    return combined_data

In [7]:
def join_frogs(frogs, data):
    """Collects the data for each frog location and joins it onto the frog data 

    Arguments:
    frogs -- dataframe containing the response variable along with ["decimalLongitude", "decimalLatitude", "key"]
    data -- xarray dataarray of features, indexed with geocoordinates
    """
    return frogs.merge(
        (
            data
            .rename('data')
            .sel(
                x=xr.DataArray(frogs.decimalLongitude, dims="key", coords={"key": frogs.key}), 
                y=xr.DataArray(frogs.decimalLatitude, dims="key", coords={"key": frogs.key}),
                method="nearest"
            )
            .to_dataframe()
            .assign(val = lambda x: x.iloc[:, -1])
            [['val']]
            .reset_index()
            .drop_duplicates()
            .pivot(index="key", columns="band", values="val")
            .reset_index()
        ),
        on = ['key'],
        how = 'inner'
    )

In [8]:
# Metrics to measure over time dimension
tc_metrics = {
    'mean':{'fn':np.nanmean,'params':{}}
    # 'min':{'fn':np.nanmin,'params':{}},
    # 'max':{'fn':np.nanmax,'params':{}}
}

# Date range to take
time_slice = ('2015-01-01','2019-12-31')

# Measurements to take
assets = ['tmax', 'tmin', 'ws', 'aet', 'def', 'pet', 'vap', 'pdsi', 'srad']

# Features to take, in form '<asset>_<metric>'
features = ['tmax_mean', 'tmin_mean', 'ws_mean', 'aet_mean', 'def_mean', 'pet_mean', 'vap_mean', 'pdsi_mean', 'srad_mean']

# Interpolate values to a 512x512 image
interp_dims = (512, 512)

# Read in test data

In [9]:
# Load in test coordinates
test_file = pd.read_csv(data_path+'level_1_challenge/challenge_1_submission_template.csv')
# Read in test regions
test_1_regions = []
with open(data_path+'level_1_challenge/challenge_1_test_regions.txt', 'r') as file: 
    for i, line in enumerate(file):
        if i > 0:
            test_1_regions.append(eval("("+line+")"))
# Load in regions and save as list of dictionaries.
test_regions = [{'title':i, 'bbox':bbox} for i, bbox in enumerate(test_1_regions)]
# Load in Predictors and Response Variables
test_df = pd.DataFrame()
for region in test_regions:
  filtered_frog = filter_bbox(test_file[['id', 'decimalLongitude', 'decimalLatitude']], region['bbox']).reset_index(drop=True).assign(key=lambda x: x.index)
  predictors = get_terraclimate(region['bbox'], tc_metrics, time_slice=time_slice, assets=assets, features=features)
  test_df = pd.concat([test_df,join_frogs(filtered_frog,predictors)])

test_df = test_df.drop('key',axis=1)

Loading data
Calculating mean
Loading data
Calculating mean
Loading data
Calculating mean
Loading data
Calculating mean
Loading data
Calculating mean


In [10]:
test_X = test_df.drop(['id','decimalLongitude','decimalLatitude'], axis=1)
for col in ['vap_mean','def_mean','srad_mean']:
  test_X[f'log_{col}'] = np.log1p(test_X[col])  # apply log1p on the skewed cols
  test_X.drop(col, axis=1, inplace=True)  # remove the original cols

test_X = test_X.loc[:,['aet_mean', 'pdsi_mean', 'pet_mean', 'tmax_mean', 'tmin_mean',
       'ws_mean', 'log_vap_mean', 'log_def_mean', 'log_srad_mean']]
# check the transformation result
test_X.head()

,aet_mean,pdsi_mean,pet_mean,tmax_mean,tmin_mean,ws_mean,log_vap_mean,log_def_mean,log_srad_mean
0,57.433334,-3.163333,113.699997,20.735003,10.630002,5.083333,0.753850,4.048592,5.187479
1,44.866665,-2.696666,118.250000,20.991669,10.638336,5.421667,0.751102,4.310575,5.185988
2,63.016666,-3.689999,104.833336,19.890003,9.645001,4.710001,0.742017,3.755759,5.185988
3,45.883335,-2.683333,110.949997,20.333334,9.463335,4.626666,0.719870,4.191673,5.198957
4,66.900002,-3.323333,99.333336,19.018335,9.733335,5.340000,0.757764,3.509055,5.174453


## Read in essentials

In [11]:
import joblib
# read in minmax scaler
scaler = joblib.load(data_path+'level_1_challenge/minmax_scalar')
# read in model weights
model_weights = pickle.load(open(data_path+'level_1_challenge/final_model.pkl', 'rb'))

In [32]:
# Re-Construct the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(test_X.shape[1],))) 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# this is the output node
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                640       
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 256)               33024     
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 75,009
Trainable params: 75,009
Non-trainable params: 0
_________________________________________________________________


In [33]:
# set the model weights from pickle.load()
for i,layer in enumerate(model.layers):
  layer.set_weights([model_weights[2*i],model_weights[2*i+1]])

# Predict on test data

In [36]:
test_X = scaler.transform(test_X)
preds = model.predict(test_X)>0.5

In [37]:
test_df['occurrenceStatus'] = preds
test_df = test_df[['id','decimalLongitude','decimalLatitude','occurrenceStatus']]
display(test_df.head())

,id,decimalLongitude,decimalLatitude,occurrenceStatus
0,0,145.207706,-37.917146,False
1,1,144.981501,-37.750974,False
2,2,145.348000,-37.961600,False
3,3,145.003000,-37.621300,False
4,4,145.647000,-38.498100,False
